# A) Server-by-server “thin view”

## 1) Setup (paths + safe view)

In [ ]:
import duckdb
from pathlib import Path

con = duckdb.connect()

# Low-memory settings
con.execute("PRAGMA threads=1;")
con.execute("PRAGMA preserve_insertion_order=false;")
con.execute("PRAGMA enable_object_cache=false;")
con.execute("PRAGMA memory_limit='2GB';")           # try 1GB if still unstable
con.execute("PRAGMA temp_directory='data/tmp_duckdb';")

# 2) Build paths robustly from the notebook folder
ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

BASE = ROOT / "data" / "by_server"

# IMPORTANT: your files are hive-partitioned like:
# data/all_backends/all_crossref/backend=crossref/data_0.parquet
all_backends = (BASE / "*" / "*.parquet").as_posix()
# datacite_glob = (BASE / "all_datacite" / "*" / "*.parquet").as_posix()
# openalex_glob = (BASE / "all_openalex" / "*" / "*.parquet").as_posix()

con.execute(f"""
CREATE OR REPLACE VIEW all_backends AS
SELECT * FROM read_parquet('{all_backends}', hive_partitioning=true, union_by_name=true);
""")
# con.execute(f"""
# CREATE OR REPLACE VIEW datacite AS
# SELECT * FROM read_parquet('{datacite_glob}', hive_partitioning=true, union_by_name=true);
# """)
# con.execute(f"""
# CREATE OR REPLACE VIEW openalex AS
# SELECT * FROM read_parquet('{openalex_glob}', hive_partitioning=true, union_by_name=true);
# """)

# A unified "all_rows" view
con.execute("""
CREATE OR REPLACE VIEW all_rows AS
SELECT * FROM all_backends
""")

print(con.execute("SHOW TABLES").fetchall())


In [ ]:
con.execute("""
SELECT backend, COUNT(*) AS total, COUNT(record_id) AS with_record_id
FROM (
    SELECT backend, record_id FROM all_backends
)
GROUP BY backend
""").df()


In [1]:
import duckdb
from pathlib import Path

con = duckdb.connect()

# Low-memory settings
con.execute("PRAGMA threads=1;")
con.execute("PRAGMA preserve_insertion_order=false;")
con.execute("PRAGMA enable_object_cache=false;")
con.execute("PRAGMA memory_limit='2GB';")           # try 1GB if still unstable
con.execute("PRAGMA temp_directory='data/tmp_duckdb';")

# 2) Build paths robustly from the notebook folder
ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

BASE = ROOT / "data" / "all_backends"

# IMPORTANT: your files are hive-partitioned like:
# data/all_backends/all_crossref/backend=crossref/data_0.parquet
crossref_glob = (BASE / "all_crossref" / "*" / "*.parquet").as_posix()
datacite_glob = (BASE / "all_datacite" / "*" / "*.parquet").as_posix()
openalex_glob = (BASE / "all_openalex" / "*" / "*.parquet").as_posix()

con.execute(f"""
CREATE OR REPLACE VIEW crossref AS
SELECT * FROM read_parquet('{crossref_glob}', hive_partitioning=true, union_by_name=true);
""")
con.execute(f"""
CREATE OR REPLACE VIEW datacite AS
SELECT * FROM read_parquet('{datacite_glob}', hive_partitioning=true, union_by_name=true);
""")
con.execute(f"""
CREATE OR REPLACE VIEW openalex AS
SELECT * FROM read_parquet('{openalex_glob}', hive_partitioning=true, union_by_name=true);
""")

# A unified "all_rows" view
con.execute("""
CREATE OR REPLACE VIEW all_rows AS
SELECT * FROM crossref
UNION ALL
SELECT * FROM datacite
UNION ALL
SELECT * FROM openalex;
""")

print(con.execute("SHOW TABLES").fetchall())


[('all_rows',), ('crossref',), ('datacite',), ('openalex',)]


### Quick helper: list servers so you can copy/paste names

In [2]:
server_name_df = con.execute("""
SELECT server_name, COUNT(*) AS n
FROM all_rows
GROUP BY 1
ORDER BY n DESC
LIMIT 200;
""").df()


In [3]:
server_name_df.head(60)

,server_name,n
0,arXiv,2920797
1,HAL,1320107
2,SSRN,1258958
3,Research Square,870976
4,RePEc: Research Papers in Economics,702078
5,bioRxiv,306948
6,eLife,247558
7,AgEcon Search,189671
8,ResearchGate,181231
9,Qeios,172316


In [4]:
server_name_df.tail(52)

,server_name,n
60,CERN document server,976
61,ARPHA Preprints,890
62,MetaArXiv,880
63,SportRxiv,878
64,Gates Open Research,863
65,AgriRxiv,818
66,Beilstein Archives,697
67,Covid-19 Preprints,647
68,EasyChair preprint,620
69,MarXiv,508


## 2) Choose the server you want to explore

In [5]:
SERVER = "arXiv"   # <-- change me (exact spelling must match server_name values)
BACKEND = None     # set to "crossref" or "datacite" or "openalex" if you want only one backend


## 3) Create a thin server view (casts to VARCHAR to avoid schema drift)

This avoids the COALESCE VARCHAR/BOOLEAN error and drops heavy JSON columns.

In [6]:
def make_server_view(server_name: str, backend: str | None = None):
    backend_filter = ""
    if backend:
        backend_filter = f"AND backend = '{backend}'"

    con.execute(f"""
    CREATE OR REPLACE VIEW server_thin AS
    SELECT
      CAST(record_id AS VARCHAR)           AS record_id,
      CAST(server_name AS VARCHAR)         AS server_name,
      CAST(backend AS VARCHAR)             AS backend,

      CAST(doi AS VARCHAR)                 AS doi,
      CAST(doi_url AS VARCHAR)             AS doi_url,
      CAST(landing_page_url AS VARCHAR)    AS landing_page_url,

      CAST(version_label AS VARCHAR)       AS version_label,

      -- Relationships (keep these for true version links)
      CAST(relations_json AS VARCHAR)       AS relations_json,
      CAST(raw_relationships_json AS VARCHAR)       AS raw_relationships_json,
      CAST(is_version_of AS VARCHAR)       AS is_version_of,      -- keep as text; we’ll interpret later
      CAST(version_of_ids_json AS VARCHAR) AS version_of_ids_json,
      CAST(is_preprint_of AS VARCHAR)      AS is_preprint_of,
      CAST(published_version_ids_json AS VARCHAR) AS published_version_ids_json,

      -- Dates (helpful for temporal patterns)
      CAST(date_posted AS VARCHAR)         AS date_posted,
      CAST(date_published AS VARCHAR)      AS date_published,
      CAST(date_published_online AS VARCHAR)      AS date_published_online,
      CAST(date_issued AS VARCHAR)         AS date_issued,
      CAST(date_deposited AS VARCHAR)      AS date_deposited,
      CAST(date_indexed AS VARCHAR)        AS date_indexed,
      CAST(date_created AS VARCHAR)        AS date_created,
      CAST(date_registered AS VARCHAR)     AS date_registered,
      CAST(date_updated AS VARCHAR)        AS date_updated,
      CAST(publication_year AS VARCHAR)    AS publication_year
    FROM all_rows
    WHERE server_name = '{server_name}'
    {backend_filter};
    """)

make_server_view(SERVER, BACKEND)
con.execute("SELECT COUNT(*) AS n FROM server_thin").df()


,n
0,2920797


In [7]:
data = con.execute("SELECT * FROM server_thin").df()
data.head()

,record_id,server_name,backend,doi,doi_url,landing_page_url,version_label,relations_json,raw_relationships_json,is_version_of,...,date_posted,date_published,date_published_online,date_issued,date_deposited,date_indexed,date_created,date_registered,date_updated,publication_year
0,datacite::10.48550/arxiv.2201.00002,arXiv,datacite,10.48550/arxiv.2201.00002,https://doi.org/10.48550/arxiv.2201.00002,https://arxiv.org/abs/2201.00002,1,"[{""relatedIdentifier"": ""10.1088/1361-6544/ac48...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2025-05-28,2022
1,datacite::10.48550/arxiv.2201.00005,arXiv,datacite,10.48550/arxiv.2201.00005,https://doi.org/10.48550/arxiv.2201.00005,https://arxiv.org/abs/2201.00005,1,[],"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-01-12,2022
2,datacite::10.48550/arxiv.2201.00006,arXiv,datacite,10.48550/arxiv.2201.00006,https://doi.org/10.48550/arxiv.2201.00006,https://arxiv.org/abs/2201.00006,3,[],"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2024-05-03,2022
3,datacite::10.48550/arxiv.2201.00018,arXiv,datacite,10.48550/arxiv.2201.00018,https://doi.org/10.48550/arxiv.2201.00018,https://arxiv.org/abs/2201.00018,1,"[{""relatedIdentifier"": ""10.21468/scipostphys.1...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-11-03,2022
4,datacite::10.48550/arxiv.2201.00019,arXiv,datacite,10.48550/arxiv.2201.00019,https://doi.org/10.48550/arxiv.2201.00019,https://arxiv.org/abs/2201.00019,2,"[{""relatedIdentifier"": ""10.1007/jhep06(2022)11...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-07-07,2022


In [8]:
data.head(60)

,record_id,server_name,backend,doi,doi_url,landing_page_url,version_label,relations_json,raw_relationships_json,is_version_of,...,date_posted,date_published,date_published_online,date_issued,date_deposited,date_indexed,date_created,date_registered,date_updated,publication_year
0,datacite::10.48550/arxiv.2201.00002,arXiv,datacite,10.48550/arxiv.2201.00002,https://doi.org/10.48550/arxiv.2201.00002,https://arxiv.org/abs/2201.00002,1,"[{""relatedIdentifier"": ""10.1088/1361-6544/ac48...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2025-05-28,2022
1,datacite::10.48550/arxiv.2201.00005,arXiv,datacite,10.48550/arxiv.2201.00005,https://doi.org/10.48550/arxiv.2201.00005,https://arxiv.org/abs/2201.00005,1,[],"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-01-12,2022
2,datacite::10.48550/arxiv.2201.00006,arXiv,datacite,10.48550/arxiv.2201.00006,https://doi.org/10.48550/arxiv.2201.00006,https://arxiv.org/abs/2201.00006,3,[],"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2024-05-03,2022
3,datacite::10.48550/arxiv.2201.00018,arXiv,datacite,10.48550/arxiv.2201.00018,https://doi.org/10.48550/arxiv.2201.00018,https://arxiv.org/abs/2201.00018,1,"[{""relatedIdentifier"": ""10.21468/scipostphys.1...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-11-03,2022
4,datacite::10.48550/arxiv.2201.00019,arXiv,datacite,10.48550/arxiv.2201.00019,https://doi.org/10.48550/arxiv.2201.00019,https://arxiv.org/abs/2201.00019,2,"[{""relatedIdentifier"": ""10.1007/jhep06(2022)11...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-07-07,2022
5,datacite::10.48550/arxiv.2201.00020,arXiv,datacite,10.48550/arxiv.2201.00020,https://doi.org/10.48550/arxiv.2201.00020,https://arxiv.org/abs/2201.00020,3,"[{""relatedIdentifier"": ""10.1038/s41467-023-380...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2023-05-26,2022
6,datacite::10.48550/arxiv.2201.00021,arXiv,datacite,10.48550/arxiv.2201.00021,https://doi.org/10.48550/arxiv.2201.00021,https://arxiv.org/abs/2201.00021,3,"[{""relatedIdentifier"": ""10.1051/0004-6361/2021...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-04-12,2022
7,datacite::10.48550/arxiv.2201.00022,arXiv,datacite,10.48550/arxiv.2201.00022,https://doi.org/10.48550/arxiv.2201.00022,https://arxiv.org/abs/2201.00022,2,"[{""relatedIdentifier"": ""10.3847/2041-8213/ac64...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-07-07,2022
8,datacite::10.48550/arxiv.2201.00023,arXiv,datacite,10.48550/arxiv.2201.00023,https://doi.org/10.48550/arxiv.2201.00023,https://arxiv.org/abs/2201.00023,1,"[{""relatedIdentifier"": ""10.1103/physrevd.105.1...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-11-10,2022
9,datacite::10.48550/arxiv.2201.00030,arXiv,datacite,10.48550/arxiv.2201.00030,https://doi.org/10.48550/arxiv.2201.00030,https://arxiv.org/abs/2201.00030,2,"[{""relatedIdentifier"": ""10.1103/physrevd.105.1...","{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",,...,None,None,None,None,None,None,2022-01-04,2022-01-04,2022-05-26,2022


In [9]:
data.count()

record_id                     2920797
server_name                   2920797
backend                       2920797
doi                           2920797
doi_url                       2920797
landing_page_url              2920797
version_label                 2920797
relations_json                2920797
raw_relationships_json        2920797
is_version_of                 2920797
version_of_ids_json                 0
is_preprint_of                2920797
published_version_ids_json          0
date_posted                         0
date_published                      0
date_published_online               0
date_issued                         0
date_deposited                      0
date_indexed                        0
date_created                  2920797
date_registered               2920797
date_updated                  2920797
publication_year              2920797
dtype: int64

In [10]:
data[['doi','relations_json']]

,doi,relations_json
0,10.48550/arxiv.2201.00002,"[{""relatedIdentifier"": ""10.1088/1361-6544/ac48..."
1,10.48550/arxiv.2201.00005,[]
2,10.48550/arxiv.2201.00006,[]
3,10.48550/arxiv.2201.00018,"[{""relatedIdentifier"": ""10.21468/scipostphys.1..."
4,10.48550/arxiv.2201.00019,"[{""relatedIdentifier"": ""10.1007/jhep06(2022)11..."
...,...,...
2920792,10.48550/arxiv.2501.10584,[]
2920793,10.48550/arxiv.2501.10585,[]
2920794,10.48550/arxiv.2501.10586,[]
2920795,10.48550/arxiv.2501.10587,"[{""relatedIdentifier"": ""10.1088/1475-7516/2025..."


In [11]:
data['relations_json'][1044404]

'[]'

#### 

In [12]:
vvv

NameError: name 'vvv' is not defined

# B) Explore versioning signals for ONLY that server

## 1) Build a clean version detection table for that server

This outputs: record_id -> version_signal_reasons (pipe-separated reasons).

In [ ]:
con.execute(r"""
CREATE OR REPLACE TABLE server_version_signals AS
WITH base AS (
  SELECT
    record_id,
    server_name,
    backend,

    lower(coalesce(doi, '')) AS doi_l,
    lower(coalesce(landing_page_url, '')) AS landing_l,
    lower(coalesce(version_label, '')) AS vlabel_l,

    -- interpret is_version_of safely from text
    CASE
      WHEN lower(coalesce(is_version_of,'')) IN ('true','t','1','yes') THEN TRUE
      ELSE FALSE
    END AS is_version_of_bool
  FROM server_thin
),
flags AS (
  SELECT
    *,
    regexp_matches(doi_l, '[-._]v[0-9]{1,3}([^0-9]|$)') AS doi_sep_vN,
    regexp_matches(doi_l, '[-._][0-9]{1,3}([^0-9]|$)')  AS doi_sep_N,

    regexp_matches(landing_l, '([?&](version|ver|v)=([0-9]{1,3}))') AS landing_param_version,
    regexp_matches(landing_l, '(/v[0-9]{1,3})(/|$)')                AS landing_path_vN,

    (vlabel_l <> '' AND vlabel_l <> 'null') AS version_label_present,

    (is_version_of_bool) AS metadata_is_version_of
  FROM base
),
reasons AS (
  SELECT
    record_id,
    server_name,
    backend,

    -- Build a pipe-separated reason string
    trim(both '|' from
      (CASE WHEN doi_sep_vN THEN '|doi_sep_vN' ELSE '' END) ||
      (CASE WHEN doi_sep_N THEN '|doi_sep_N' ELSE '' END) ||
      (CASE WHEN landing_param_version THEN '|landing_param_version' ELSE '' END) ||
      (CASE WHEN landing_path_vN THEN '|landing_path_vN' ELSE '' END) ||
      (CASE WHEN version_label_present THEN '|metadata_version_label' ELSE '' END) ||
      (CASE WHEN metadata_is_version_of THEN '|metadata_is_version_of' ELSE '' END)
    ) AS version_signal_reasons
  FROM flags
)
SELECT
  record_id, server_name, backend,
  CASE
    WHEN version_signal_reasons = '' THEN 'no_signal'
    ELSE version_signal_reasons
  END AS version_signal_reasons
FROM reasons;
""")

con.execute("""
SELECT version_signal_reasons, COUNT(*) AS n
FROM server_version_signals
GROUP BY 1
ORDER BY n DESC;
""").df()


## 2) Inspect records with any signal (sample)

In [ ]:
con.execute("""
SELECT *
FROM server_version_signals
WHERE version_signal_reasons <> 'no_signal'
LIMIT 200;
""").df()


# C) Export only THIS server (safe, chunked)

If you want to move one server to Colab or work offline.

In [ ]:
import math
from pathlib import Path

OUT = Path(f"data/all_backends/server_exports/{SERVER}")
OUT.mkdir(parents=True, exist_ok=True)

ROWS_PER_PART = 200_000

total = con.execute("SELECT COUNT(*) FROM server_thin").fetchone()[0]
n_parts = math.ceil(total / ROWS_PER_PART)
print("rows:", total, "parts:", n_parts)

for i in range(n_parts):
    offset = i * ROWS_PER_PART
    out_path = OUT / f"server_thin_part_{i:06d}.parquet"
    con.execute(f"""
        COPY (
            SELECT * FROM server_thin
            LIMIT {ROWS_PER_PART} OFFSET {offset}
        )
        TO '{out_path.as_posix()}'
        (FORMAT PARQUET);
    """)
print("done ->", OUT)
